In [41]:
from pathlib import Path
import subprocess
import os
import zipfile
import hashlib

current_working_dir = Path.cwd()
steghide_images_dir = current_working_dir / Path('steghide_images')  #Combine the paths. This is using operator overloading for the / operator
zipped_images_dir = current_working_dir / Path('zipped_images')
extracted_images_dir = current_working_dir / Path('extracted_images')
experiment_results_dir = current_working_dir / Path('experiments_results')

zipped_images_dir.mkdir(parents=True, exist_ok=True)
print("Created the zipped images directory")
extracted_images_dir.mkdir(parents=True, exist_ok=True)
print("Created the extracted images directory")
experiment_results_dir.mkdir(parents=True, exist_ok=True)
print("Created the experiment results directory")

Created the zipped images directory
Created the extracted images directory
Created the experiment results directory


In [42]:

def calculate_file_hash(file_name):
    hash_obj = hashlib.sha256()
    with open(file_name, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_obj.update(chunk)
    return hash_obj.hexdigest()

In [44]:
image_directories = []

for directory_item in os.listdir(steghide_images_dir):
    if not os.path.isfile(directory_item):
        image_directories.append(steghide_images_dir / Path(directory_item))
        
print("These are the image directories inside of: " + str(image_directories))

zipped_test_results_file = open(str(experiment_results_dir / Path('zipped_test_results.txt')), 'w')


for image_directory in image_directories:
    for image_file in os.listdir(image_directory):
        original_filename = str(image_directory / Path(image_file))
        zipped_filename = str(zipped_images_dir / Path(image_file).with_suffix('.zip'))
        extracted_filename = str(extracted_images_dir / Path(image_file))
        
        # print("Zipping this image file: " + image_file)
        # print("This the original_filename: " + original_filename)
        # print("This is the zipped filename: " + zipped_filename)
        # print("This is the extracted filename: " + extracted_filename)

        with zipfile.ZipFile(zipped_filename, 'w') as zip_ref:
            zip_ref.write(original_filename, arcname=original_filename)
        with zipfile.ZipFile(zipped_filename, 'r') as zip_ref:
            file_data = zip_ref.read(original_filename[1:])
            with open(extracted_filename, 'wb') as new_extracted_file:
                new_extracted_file.write(file_data)
        
        original_file_hash = calculate_file_hash(original_filename)
        extracted_file_hash = calculate_file_hash(extracted_filename)
        
        zipped_test_results_file.write(f"File: {image_file}\n")
        zipped_test_results_file.write(f"Original hash: {original_file_hash}\n")
        zipped_test_results_file.write(f"Extract  hash: {extracted_file_hash}\n")
        if original_file_hash != extracted_file_hash:
            zipped_test_results_file.write(f"This file is not the same before and after compression: {image_file}\n")
        else:
            zipped_test_results_file.write("The images are matching\n")
        zipped_test_results_file.write("\n")

        
        

These are the image directories inside of: [PosixPath('/home/rafad900/covert-channel-project/steghide_images/cars'), PosixPath('/home/rafad900/covert-channel-project/steghide_images/dogs'), PosixPath('/home/rafad900/covert-channel-project/steghide_images/bike'), PosixPath('/home/rafad900/covert-channel-project/steghide_images/horses'), PosixPath('/home/rafad900/covert-channel-project/steghide_images/cats'), PosixPath('/home/rafad900/covert-channel-project/steghide_images/human')]
